In [13]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import datetime
import csv
import datetime
from operator import sub
import xgboost as xgb
from sklearn import preprocessing, ensemble
import os
#print(os.listdir("../input"))
from sklearn import tree
import seaborn as sns
from collections import defaultdict
# Any results you write to the current directory are saved as output.

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
data_path = "/content/drive/My Drive/recommendation bank/"

In [ ]:
#### Mapping all categorical variables with corresponding values

In [4]:
mapping_dict = {
'ind_empleado'  : {-99:0, 'N':1, 'B':2, 'F':3, 'A':4, 'S':5},
'sexo'          : {'V':0, 'H':1, -99:2},
'ind_nuevo'     : {'0':0, '1':1, -99:2},
'indrel'        : {'1':0, '99':1, -99:2},
'indrel_1mes'   : {-99:0, '1.0':1, '1':1, '2.0':2, '2':2, '3.0':3, '3':3, '4.0':4, '4':4, 'P':5},
'tiprel_1mes'   : {-99:0, 'I':1, 'A':2, 'P':3, 'R':4, 'N':5},
'indresi'       : {-99:0, 'S':1, 'N':2},
'indext'        : {-99:0, 'S':1, 'N':2},
'conyuemp'      : {-99:0, 'S':1, 'N':2},
'indfall'       : {-99:0, 'S':1, 'N':2},
'tipodom'       : {-99:0, '1':1},
'ind_actividad_cliente' : {'0':0, '1':1, -99:2},
'segmento'      : {'02 - PARTICULARES':0, '03 - UNIVERSITARIO':1, '01 - TOP':2, -99:2},
'pais_residencia' : {'LV': 102, 'BE': 12, 'BG': 50, 'BA': 61, 'BM': 117, 'BO': 62, 'JP': 82, 'JM': 116, 'BR': 17, 'BY': 64, 'BZ': 113, 'RU': 43, 'RS': 89, 'RO': 41, 'GW': 99, 'GT': 44, 'GR': 39, 'GQ': 73, 'GE': 78, 'GB': 9, 'GA': 45, 'GN': 98, 'GM': 110, 'GI': 96, 'GH': 88, 'OM': 100, 'HR': 67, 'HU': 106, 'HK': 34, 'HN': 22, 'AD': 35, 'PR': 40, 'PT': 26, 'PY': 51, 'PA': 60, 'PE': 20, 'PK': 84, 'PH': 91, 'PL': 30, 'EE': 52, 'EG': 74, 'ZA': 75, 'EC': 19, 'AL': 25, 'VN': 90, 'ET': 54, 'ZW': 114, 'ES': 0, 'MD': 68, 'UY': 77, 'MM': 94, 'ML': 104, 'US': 15, 'MT': 118, 'MR': 48, 'UA': 49, 'MX': 16, 'IL': 42, 'FR': 8, 'MA': 38, 'FI': 23, 'NI': 33, 'NL': 7, 'NO': 46, 'NG': 83, 'NZ': 93, 'CI': 57, 'CH': 3, 'CO': 21, 'CN': 28, 'CM': 55, 'CL': 4, 'CA': 2, 'CG': 101, 'CF': 109, 'CD': 112, 'CZ': 36, 'CR': 32, 'CU': 72, 'KE': 65, 'KH': 95, 'SV': 53, 'SK': 69, 'KR': 87, 'KW': 92, 'SN': 47, 'SL': 97, 'KZ': 111, 'SA': 56, 'SG': 66, 'SE': 24, 'DO': 11, 'DJ': 115, 'DK': 76, 'DE': 10, 'DZ': 80, 'MK': 105, -99: 1, 'LB': 81, 'TW': 29, 'TR': 70, 'TN': 85, 'LT': 103, 'LU': 59, 'TH': 79, 'TG': 86, 'LY': 108, 'AE': 37, 'VE': 14, 'IS': 107, 'IT': 18, 'AO': 71, 'AR': 13, 'AU': 63, 'AT': 6, 'IN': 31, 'IE': 5, 'QA': 58, 'MZ': 27},
'canal_entrada' : {'013': 49, 'KHP': 160, 'KHQ': 157, 'KHR': 161, 'KHS': 162, 'KHK': 10, 'KHL': 0, 'KHM': 12, 'KHN': 21, 'KHO': 13, 'KHA': 22, 'KHC': 9, 'KHD': 2, 'KHE': 1, 'KHF': 19, '025': 159, 'KAC': 57, 'KAB': 28, 'KAA': 39, 'KAG': 26, 'KAF': 23, 'KAE': 30, 'KAD': 16, 'KAK': 51, 'KAJ': 41, 'KAI': 35, 'KAH': 31, 'KAO': 94, 'KAN': 110, 'KAM': 107, 'KAL': 74, 'KAS': 70, 'KAR': 32, 'KAQ': 37, 'KAP': 46, 'KAW': 76, 'KAV': 139, 'KAU': 142, 'KAT': 5, 'KAZ': 7, 'KAY': 54, 'KBJ': 133, 'KBH': 90, 'KBN': 122, 'KBO': 64, 'KBL': 88, 'KBM': 135, 'KBB': 131, 'KBF': 102, 'KBG': 17, 'KBD': 109, 'KBE': 119, 'KBZ': 67, 'KBX': 116, 'KBY': 111, 'KBR': 101, 'KBS': 118, 'KBP': 121, 'KBQ': 62, 'KBV': 100, 'KBW': 114, 'KBU': 55, 'KCE': 86, 'KCD': 85, 'KCG': 59, 'KCF': 105, 'KCA': 73, 'KCC': 29, 'KCB': 78, 'KCM': 82, 'KCL': 53, 'KCO': 104, 'KCN': 81, 'KCI': 65, 'KCH': 84, 'KCK': 52, 'KCJ': 156, 'KCU': 115, 'KCT': 112, 'KCV': 106, 'KCQ': 154, 'KCP': 129, 'KCS': 77, 'KCR': 153, 'KCX': 120, 'RED': 8, 'KDL': 158, 'KDM': 130, 'KDN': 151, 'KDO': 60, 'KDH': 14, 'KDI': 150, 'KDD': 113, 'KDE': 47, 'KDF': 127, 'KDG': 126, 'KDA': 63, 'KDB': 117, 'KDC': 75, 'KDX': 69, 'KDY': 61, 'KDZ': 99, 'KDT': 58, 'KDU': 79, 'KDV': 91, 'KDW': 132, 'KDP': 103, 'KDQ': 80, 'KDR': 56, 'KDS': 124, 'K00': 50, 'KEO': 96, 'KEN': 137, 'KEM': 155, 'KEL': 125, 'KEK': 145, 'KEJ': 95, 'KEI': 97, 'KEH': 15, 'KEG': 136, 'KEF': 128, 'KEE': 152, 'KED': 143, 'KEC': 66, 'KEB': 123, 'KEA': 89, 'KEZ': 108, 'KEY': 93, 'KEW': 98, 'KEV': 87, 'KEU': 72, 'KES': 68, 'KEQ': 138, -99: 6, 'KFV': 48, 'KFT': 92, 'KFU': 36, 'KFR': 144, 'KFS': 38, 'KFP': 40, 'KFF': 45, 'KFG': 27, 'KFD': 25, 'KFE': 148, 'KFB': 146, 'KFC': 4, 'KFA': 3, 'KFN': 42, 'KFL': 34, 'KFM': 141, 'KFJ': 33, 'KFK': 20, 'KFH': 140, 'KFI': 134, '007': 71, '004': 83, 'KGU': 149, 'KGW': 147, 'KGV': 43, 'KGY': 44, 'KGX': 24, 'KGC': 18, 'KGN': 11}
}
cat_cols = list(mapping_dict.keys())

In [5]:
#### Seprating columns to predict
('Savings_Acct', 'Guarantees', 'Cur_Acct', 'Derivative_Acct', 'Payroll_Acct','Junior_Acct', 'Mas_Particular_Acct', 'Particular_Acct', 'Particular_Plus_Acct', 'Short_Term_Deposits','Med_Term_Deposits', 'Long_Term_Deposits', 'e-Acct', 'Funds', 'Mortgage', 'Pension1', 'Loans','Taxes', 'Credit_Card', 'Securities', 'Home_Acct', 'Payroll', 'Pension2', 'Direct_Debit')

('Savings_Acct',
 'Guarantees',
 'Cur_Acct',
 'Derivative_Acct',
 'Payroll_Acct',
 'Junior_Acct',
 'Mas_Particular_Acct',
 'Particular_Acct',
 'Particular_Plus_Acct',
 'Short_Term_Deposits',
 'Med_Term_Deposits',
 'Long_Term_Deposits',
 'e-Acct',
 'Funds',
 'Mortgage',
 'Pension1',
 'Loans',
 'Taxes',
 'Credit_Card',
 'Securities',
 'Home_Acct',
 'Payroll',
 'Pension2',
 'Direct_Debit')

In [ ]:
#### Not considering first 2 columns as there were 0 purchase in last month( Saving and Guarantees)(Found during EDA of data)

In [30]:
target_cols = ['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1','ind_nom_pens_ult1','ind_recibo_ult1']
target_cols = target_cols[2:]

In [ ]:
### While reading each row in file getting corresponding present account user already have

##### Function to gather the values of product columns--filling in Null and "NA" so that  compare which were added. 
##### This is require to reduce the dataset to only the customers who added a product in a subsequent month. 

In [17]:
def getTarget(row):
	tlist = []
	for col in target_cols:
		if row[col].strip() in ['', 'NA']:
			target = 0
		else:
			target = int(float(row[col]))
		tlist.append(target)
	return tlist

In [ ]:
### Getting index of corresponding row and columns

In [18]:
def getIndex(row, col):
	val = row[col].strip()
	if val not in ['','NA']:
		ind = mapping_dict[col][val]
	else:
		ind = mapping_dict[col][-99]
	return ind

In [ ]:

Impute N/A values and recode Age of customer
Grouping 20, as 20 years old and over 90 as 90 to reduce extreme values. 
Customers unlikely to be adding many products in the tails. 
With 8 missing values in Training set, setting to 40---value most comming in Training set for customers 
that actually added products (younger people seem to be customers, but not ones that end up adding new products)

In [19]:
def getAge(row):
	mean_age = 40.
	min_age = 20.
	max_age = 90.
	range_age = max_age - min_age
	age = row['age'].strip()
	if age == 'NA' or age == '':
		age = mean_age
	else:
		age = float(age)
		if age < min_age:
			age = min_age
		elif age > max_age:
			age = max_age
	return round( (age - min_age) / range_age, 4)

Impute N/A values and recode Antiquedad: (Seniority)
Small number and will give them the least seniorty(zero)rather than remove them. 

In [20]:
def getCustSeniority(row):
	min_value = 0.
	max_value = 256.
	range_value = max_value - min_value
	missing_value = 0.
	cust_seniority = row['antiguedad'].strip()
	if cust_seniority == 'NA' or cust_seniority == '':
		cust_seniority = missing_value
	else:
		cust_seniority = float(cust_seniority)
		if cust_seniority < min_value:
			cust_seniority = min_value
		elif cust_seniority > max_value:
			cust_seniority = max_value
	return round((cust_seniority-min_value) / range_value, 4)

impute income(Renta)-Training Data Set code highest earners to 1.5M

In [21]:
def getRent(row):
	min_value = 0.
	max_value = 1500000.
	range_value = max_value - min_value
	missing_value = 101850.
	rent = row['renta'].strip()
	if rent == 'NA' or rent == '':
		rent = missing_value
	else:
		rent = float(rent)
		if rent < min_value:
			rent = min_value
		elif rent > max_value:
			rent = max_value
	return round((rent-min_value) / range_value, 6)

##### This function allows to create a data frame by only any given month to the subsequent month, allowing  model only rows 
##### which the users added a product.

In [22]:

def processData(in_file_name, cust_dict):
	x_vars_list = []
	y_vars_list = []
	for row in csv.DictReader(in_file_name):
		# use only the four months
		if row['fecha_dato'] not in ['2015-05-28', '2015-06-28', '2016-05-28', '2016-06-28']:
			continue

		cust_id = int(row['ncodpers'])
		if row['fecha_dato'] in ['2015-05-28', '2016-05-28']:	
			target_list = getTarget(row)
			cust_dict[cust_id] =  target_list[:]
			continue

		x_vars = []
		for col in cat_cols:

			x_vars.append( getIndex(row, col) )
			x_vars.append( getAge(row) )
			x_vars.append( getCustSeniority(row) )
			x_vars.append( getRent(row) )

		if row['fecha_dato'] == '2016-06-28':
			prev_target_list = cust_dict.get(cust_id, [0]*22)
			x_vars_list.append(x_vars + prev_target_list)
		elif row['fecha_dato'] == '2015-06-28':
			prev_target_list = cust_dict.get(cust_id, [0]*22)
			target_list = getTarget(row)
			new_products = [max(x1 - x2,0) for (x1, x2) in zip(target_list, prev_target_list)]
			if sum(new_products) > 0:
				for ind, prod in enumerate(new_products):
					if prod>0:
						assert len(prev_target_list) == 22
						x_vars_list.append(x_vars+prev_target_list)
						y_vars_list.append(ind)

	return x_vars_list, y_vars_list, cust_dict

In [31]:
train_file =  open(data_path + "train_ver2.0.csv")

In [25]:
train_file

<_io.TextIOWrapper name='/content/drive/My Drive/recommendation bank/train_ver2.0.csv' mode='r' encoding='UTF-8'>

##### Select only customers with product in May 2015 that added one in June 2015 to match seasonality with the requirement forJune predictions. 

##### Create a Y column based on the products using numbers to represent each possible product 

In [32]:
x_vars_list, y_vars_list, cust_dict = processData(train_file, {})

In [33]:
cust_dict

{1061260: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1061283: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1061284: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1061336: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1061286: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1061290: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1061288: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1061289: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1061281: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1061340: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1061291: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1061292: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1061293: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [34]:
	train_X = np.array(x_vars_list)
	train_y = np.array(y_vars_list)
	print(np.unique(train_y))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]


In [ ]:
	del x_vars_list, y_vars_list
	

In [35]:
train_file.close()

In [36]:
train_X

array([[1.    , 0.1143, 0.1328, ..., 0.    , 0.    , 0.    ],
       [1.    , 0.1143, 0.1328, ..., 0.    , 0.    , 0.    ],
       [1.    , 0.2429, 0.1328, ..., 0.    , 0.    , 1.    ],
       ...,
       [1.    , 0.6571, 0.4648, ..., 0.    , 0.    , 0.    ],
       [1.    , 0.4857, 0.4648, ..., 0.    , 0.    , 0.    ],
       [1.    , 0.3714, 0.2305, ..., 1.    , 1.    , 1.    ]])

In [37]:
	print(train_X.shape, train_y.shape)


(45679, 82) (45679,)


In [38]:
test_file = open(data_path + "test_ver2.0.csv")

In [39]:
x_vars_list, y_vars_list, cust_dict = processData(test_file, cust_dict)

In [40]:
cust_dict

{1061260: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1061283: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1061284: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1061336: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1061286: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1061290: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1061288: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1061289: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1061281: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1061340: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1061291: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1061292: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1061293: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [41]:
test_X = np.array(x_vars_list)

In [42]:
	#del x_vars_list
	test_file.close()
	print(test_X.shape)

(929615, 82)


multi:softprob: same as softmax, but output a vector of ndata * nclass, which can be further reshaped to ndata * nclass matrix. The result contains predicted probability of each data point belonging to each class.

In [43]:
def runXGB(train_X, train_y, seed_val=0):
	param = {}
	param['objective'] = 'multi:softprob'
	param['eta'] = 0.05
	param['max_depth'] = 8
	param['silent'] = 1
	param['num_class'] = 22
	param['eval_metric'] = "mlogloss"
	param['min_child_weight'] = 1
	param['subsample'] = 0.7
	param['colsample_bytree'] = 0.7
	param['seed'] = seed_val
	num_rounds = 50

	plst = list(param.items())
	xgtrain = xgb.DMatrix(train_X, label=train_y)
	model = xgb.train(plst, xgtrain, num_rounds)	
	return model

In [44]:
import xgboost as xgb
import sys
import math
 
 
class XGBoostClassifier():
    def __init__(self, num_boost_round=10, **params):
        self.clf = None
        self.num_boost_round = num_boost_round
        self.params = params
        self.params.update({'objective': 'multi:softprob'})
 
    def fit(self, X, y, num_boost_round=None):
        num_boost_round = num_boost_round or self.num_boost_round
        self.label2num = dict((label, i) for i, label in enumerate(sorted(set(y))))
        dtrain = xgb.DMatrix(X, label=[self.label2num[label] for label in y])
        self.clf = xgb.train(params=self.params, dtrain=dtrain, num_boost_round=num_boost_round)
 
    def predict(self, X):
        num2label = dict((i, label)for label, i in self.label2num.items())
        Y = self.predict_proba(X)
        y = np.argmax(Y, axis=1)
        return np.array([num2label[i] for i in y])
 
    def predict_proba(self, X):
        dtest = xgb.DMatrix(X)
        return self.clf.predict(dtest)
 
    def score(self, X, y):
        Y = self.predict_proba(X)
        return 1 / logloss(y, Y)
 
    def get_params(self, deep=True):
        return self.params
 
    def set_params(self, **params):
        if 'num_boost_round' in params:
            self.num_boost_round = params.pop('num_boost_round')
        if 'objective' in params:
            del params['objective']
        self.params.update(params)
        return self
    
    
def logloss(y_true, Y_pred):
    label2num = dict((name, i) for i, name in enumerate(sorted(set(y_true))))
    return -1 * sum(math.log(y[label2num[label]]) if y[label2num[label]] > 0 else -np.inf for y, label in zip(Y_pred, y_true)) / len(Y_pred)

In [45]:
from sklearn.model_selection import GridSearchCV

In [46]:
def main():
    clf = XGBoostClassifier(
        eval_metric = 'logloss',
        num_class = 22,
        nthread = 4,
        eta = 0.07,
        num_boost_round = 10,
        max_depth = 9,
        min_child_weight = 1,
        subsample = 0.85,
        colsample_bytree = 0.8,
        silent = 1,
        reg_alpha=0.07,
        gamma = 0.9
        )
    parameters = {
        'gamma':[0.6, 0.7,0.8,0.9,1]
        #'num_boost_round': [50, 100, 150],
        #'eta': [0.06,0.065,0.07],
        #'reg_alpha':[0.067,0.07, 0.072,0.075]
        #'max_depth': [7,8,9],
        #'min_child_weight':[1,2,3]
        #'subsample': [0.9, 0.8, 0.85, 0.95],
        #'colsample_bytree': [0.8,0.85, 0.9, 0.95],
    }
    clf = GridSearchCV(clf, parameters, n_jobs=1, cv=2)
    print("Fitting the model")
    clf.fit(train_X,train_y)
    print("Training model completed")
    #best_parameters, score, _ = max(clf.make_scorer, key=lambda x: x[1])
    #print(score)
    #for param_name in sorted(best_parameters.keys()):
    #    print("%s: %r" % (param_name, best_parameters[param_name]))
                
    print(clf.predict(test_X))


if __name__ == '__main__':
    main()

Fitting the model
Training model completed
[21 21 21 ... 21 21  0]


In [47]:
print("Building model..")
model = runXGB(train_X, train_y, seed_val=0)

Building model..


In [48]:
print("Predicting..")

Predicting..


In [49]:
xgtest = xgb.DMatrix(test_X)

In [50]:
xgtest

In [58]:
xgtrain = xgb.DMatrix(train_X)

In [59]:
pred=model.predict(xgtrain)

##### Predict probabilities of each class label so that later rank order them and choose the top 7 to RECOMMEND

In [51]:
preds = model.predict(xgtest)

In [64]:
from sklearn.metrics import log_loss

In [62]:
pred.shape

(45679, 22)

In [60]:
print(log_loss(train_y, pred))

1.3537151035592676


In [ ]:
preds

array([[0.02056875, 0.01093911, 0.02375693, ..., 0.04007149, 0.03193856,
        0.5167402 ],
       [0.02077109, 0.0108031 , 0.0420744 , ..., 0.06505527, 0.06349229,
        0.48001245],
       [0.02045626, 0.01088965, 0.05260789, ..., 0.1011353 , 0.10014366,
        0.51803064],
       ...,
       [0.01875186, 0.01111051, 0.01772064, ..., 0.03483165, 0.03506904,
        0.4236012 ],
       [0.02077133, 0.0110365 , 0.03344002, ..., 0.04219763, 0.04155631,
        0.49873826],
       [0.39169514, 0.01325551, 0.03004788, ..., 0.01674626, 0.09223143,
        0.09390797]], dtype=float32)

In [ ]:
del test_X, xgtest

In [ ]:
print("Getting the top products..")

Getting the top products..


In [28]:
target_cols = np.array(target_cols)

target_cols

array(['ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1',
       'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1',
       'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1',
       'ind_dela_fin_ult1', 'ind_ecue_fin_ult1', 'ind_fond_fin_ult1',
       'ind_hip_fin_ult1', 'ind_plan_fin_ult1', 'ind_pres_fin_ult1',
       'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1',
       'ind_viv_fin_ult1', 'ind_nomina_ult1', 'ind_nom_pens_ult1',
       'ind_recibo_ult1'], dtype='<U17')

In [ ]:
preds = np.argsort(preds, axis=1)
preds

array([[ 3,  7,  1, ..., 19, 15, 21],
       [ 3,  6, 18, ..., 19, 15, 21],
       [ 3,  6, 18, ..., 20, 19, 21],
       ...,
       [ 3, 18,  7, ...,  9, 16, 21],
       [ 3, 18,  7, ..., 15, 16, 21],
       [ 3, 18,  7, ..., 20, 21,  0]])

##### Choose the 7 most likely product to be added

In [ ]:
preds = np.fliplr(preds)[:,:7]
preds

array([[21, 15, 19, ..., 16,  9, 10],
       [21, 15, 19, ..., 16,  2, 10],
       [21, 19, 20, ..., 16,  0, 15],
       ...,
       [21, 16,  9, ..., 20, 19, 10],
       [21, 16, 15, ..., 20,  2, 10],
       [ 0, 21, 20, ...,  9, 15, 16]])

##### merge predictions with customer IDs and save file
##### join customer ID (ncodpers) from prediction set with predictions

In [ ]:
test_id = np.array(pd.read_csv(data_path + "test_ver2.0.csv", usecols=['ncodpers'])['ncodpers'])
final_preds = [" ".join(list(target_cols[pred])) for pred in preds]
out_df = pd.DataFrame({'ncodpers':test_id, 'added_products':final_preds})

In [ ]:
test_id

array([  15889, 1170544, 1170545, ...,  660240,  660243,  660248])

In [ ]:
final_preds[0]

'ind_recibo_ult1 ind_reca_fin_ult1 ind_nomina_ult1 ind_nom_pens_ult1 ind_tjcr_fin_ult1 ind_dela_fin_ult1 ind_ecue_fin_ult1'

In [ ]:
out_df

,ncodpers,added_products
0,15889,ind_recibo_ult1 ind_reca_fin_ult1 ind_nomina_u...
1,1170544,ind_recibo_ult1 ind_reca_fin_ult1 ind_nomina_u...
2,1170545,ind_recibo_ult1 ind_nomina_ult1 ind_nom_pens_u...
3,1170547,ind_recibo_ult1 ind_reca_fin_ult1 ind_nom_pens...
4,1170548,ind_recibo_ult1 ind_reca_fin_ult1 ind_nom_pens...
...,...,...
929610,660237,ind_tjcr_fin_ult1 ind_reca_fin_ult1 ind_dela_f...
929611,660238,ind_cco_fin_ult1 ind_recibo_ult1 ind_tjcr_fin_...
929612,660240,ind_recibo_ult1 ind_tjcr_fin_ult1 ind_dela_fin...
929613,660243,ind_recibo_ult1 ind_tjcr_fin_ult1 ind_reca_fin...


In [ ]:
out_df.head(1)

,ncodpers,added_products
0,15889,ind_recibo_ult1 ind_reca_fin_ult1 ind_nomina_u...


In [ ]:
import operator
importance = model.get_fscore()
importance = sorted(importance.items(), key=operator.itemgetter(1))
print(importance)
print (len(importance))

[('f22', 2), ('f12', 12), ('f36', 14), ('f61', 18), ('f8', 26), ('f24', 27), ('f23', 33), ('f0', 40), ('f52', 45), ('f19', 46), ('f74', 47), ('f21', 50), ('f68', 54), ('f78', 59), ('f17', 59), ('f18', 60), ('f63', 131), ('f72', 158), ('f15', 260), ('f13', 271), ('f73', 305), ('f67', 340), ('f14', 352), ('f71', 356), ('f64', 373), ('f77', 416), ('f28', 451), ('f65', 454), ('f66', 457), ('f79', 525), ('f80', 544), ('f44', 591), ('f20', 679), ('f69', 703), ('f75', 751), ('f9', 848), ('f10', 877), ('f62', 884), ('f76', 975), ('f70', 1022), ('f81', 1069), ('f11', 1071), ('f4', 1110), ('f48', 1303), ('f60', 1336), ('f6', 2559), ('f5', 2590), ('f56', 3635), ('f7', 3885), ('f1', 8588), ('f2', 9223), ('f3', 11415)]
52


**LIGHTFM** **implementation**

In [4]:
train = pd.read_csv(data_path + "train_ver2.0.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,8,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
product_cols = ['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1',
                'ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1','ind_ctop_fin_ult1',
                'ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_dela_fin_ult1','ind_deme_fin_ult1',
                'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_nom_pens_ult1',
                'ind_nomina_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1',
                'ind_recibo_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1']

In [8]:
pip install lightfm

     |████████████████████████████████| 307kB 2.8MB/s 
  Created wheel for lightfm: filename=lightfm-1.15-cp36-cp36m-linux_x86_64.whl size=707617 sha256=d5ef4eb913ca3e4e5bba98adbae1d1f71a63b8d0a6a741680d76e20a6b4f1d38
  Stored in directory: /root/.cache/pip/wheels/eb/bb/ac/188385a5da6627956be5d9663928483b36da576149ab5b8f79
Successfully built lightfm


In [9]:
from lightfm.data import Dataset

define function for the sequential partial fitting of the model to each month

In [10]:
from lightfm import LightFM

In [6]:
dates = train.fecha_dato.unique()
dates

array(['2015-01-28', '2015-02-28', '2015-03-28', '2015-04-28',
       '2015-05-28', '2015-06-28', '2015-07-28', '2015-08-28',
       '2015-09-28', '2015-10-28', '2015-11-28', '2015-12-28',
       '2016-01-28', '2016-02-28', '2016-03-28', '2016-04-28',
       '2016-05-28'], dtype=object)

In [11]:
m1 = train.loc[train['fecha_dato']==dates[0], product_cols+['ncodpers']]
m2 = train.loc[train['fecha_dato']==dates[1], product_cols+['ncodpers']]
m3 = train.loc[train['fecha_dato']==dates[2], product_cols+['ncodpers']]
m4 = train.loc[train['fecha_dato']==dates[3], product_cols+['ncodpers']]
m5 = train.loc[train['fecha_dato']==dates[4], product_cols+['ncodpers']]
m6 = train.loc[train['fecha_dato']==dates[5], product_cols+['ncodpers']]
m7 = train.loc[train['fecha_dato']==dates[6], product_cols+['ncodpers']]
m8 = train.loc[train['fecha_dato']==dates[7], product_cols+['ncodpers']]
m9 = train.loc[train['fecha_dato']==dates[8], product_cols+['ncodpers']]
m10 = train.loc[train['fecha_dato']==dates[9], product_cols+['ncodpers']]
m11 = train.loc[train['fecha_dato']==dates[10], product_cols+['ncodpers']]
m12 = train.loc[train['fecha_dato']==dates[11], product_cols+['ncodpers']]
m13 = train.loc[train['fecha_dato']==dates[12], product_cols+['ncodpers']]
m14 = train.loc[train['fecha_dato']==dates[13], product_cols+['ncodpers']]
m15 = train.loc[train['fecha_dato']==dates[14], product_cols+['ncodpers']]
m16 = train.loc[train['fecha_dato']==dates[15], product_cols+['ncodpers']]
m17 = train.loc[train['fecha_dato']==dates[16], product_cols+['ncodpers']]

In [12]:

final_month = train[train.fecha_dato == '2016-05-28']

In [13]:
train = []

In [14]:
dataset = Dataset()

In [15]:
dataset.fit_partial(m1['ncodpers'],product_cols)

In [16]:
dataset.fit_partial(m2['ncodpers'],product_cols)
dataset.fit_partial(m3['ncodpers'],product_cols)
dataset.fit_partial(m4['ncodpers'],product_cols)
dataset.fit_partial(m5['ncodpers'],product_cols)
dataset.fit_partial(m6['ncodpers'],product_cols)
dataset.fit_partial(m7['ncodpers'],product_cols)
dataset.fit_partial(m8['ncodpers'],product_cols)
dataset.fit_partial(m9['ncodpers'],product_cols)
dataset.fit_partial(m10['ncodpers'],product_cols)
dataset.fit_partial(m11['ncodpers'],product_cols)
dataset.fit_partial(m12['ncodpers'],product_cols)
dataset.fit_partial(m13['ncodpers'],product_cols)
dataset.fit_partial(m14['ncodpers'],product_cols)
dataset.fit_partial(m15['ncodpers'],product_cols)
dataset.fit_partial(m16['ncodpers'],product_cols)
dataset.fit_partial(m17['ncodpers'],product_cols)

In [17]:
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 956645, num_items 24.


In [18]:
model = LightFM(no_components=20,learning_schedule='adadelta',loss='warp')

Probably better add the interactions for each month using a loop


In [20]:
months = [m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13,m14,m15,m16,m17]
month_weights= [.5,.5,.5,.5,1,3,.5,.5,.5,.5,1,1,1,1,1,1,2]

In [21]:
weight=0
for m in months:
    labels = m.set_index("ncodpers")
    stacked_labels = labels.stack()
    filtered_labels = stacked_labels.reset_index()
    filtered_labels['weights'] = month_weights[weight]
    filtered_labels.columns = ["ncodpers", "product", "interaction",'weights']
    interactionsdf = filtered_labels.loc[filtered_labels['interaction']==1]
    (interactions, weights) = dataset.build_interactions((interactionsdf.iloc[x,0], interactionsdf.iloc[x,1], interactionsdf.iloc[x,2])
                                                     for x in range(len(interactionsdf)))
    print(repr(interactions))
    model.fit_partial(interactions,epochs=7)
    weight += 1

<956645x24 sparse matrix of type '<class 'numpy.int32'>'
	with 1109367 stored elements in COOrdinate format>
<956645x24 sparse matrix of type '<class 'numpy.int32'>'
	with 1112733 stored elements in COOrdinate format>
<956645x24 sparse matrix of type '<class 'numpy.int32'>'
	with 1118829 stored elements in COOrdinate format>
<956645x24 sparse matrix of type '<class 'numpy.int32'>'
	with 1124795 stored elements in COOrdinate format>
<956645x24 sparse matrix of type '<class 'numpy.int32'>'
	with 1119858 stored elements in COOrdinate format>
<956645x24 sparse matrix of type '<class 'numpy.int32'>'
	with 1132501 stored elements in COOrdinate format>
<956645x24 sparse matrix of type '<class 'numpy.int32'>'
	with 1147960 stored elements in COOrdinate format>
<956645x24 sparse matrix of type '<class 'numpy.int32'>'
	with 1151798 stored elements in COOrdinate format>
<956645x24 sparse matrix of type '<class 'numpy.int32'>'
	with 1173705 stored elements in COOrdinate format>
<956645x24 sparse m

In [22]:
test = pd.read_csv(data_path + "test_ver2.0.csv",nrows=40000)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
users = test.ncodpers.values

In [24]:
users

array([  15889, 1170544, 1170545, ..., 1117299, 1117300, 1117301])

In [25]:
n_users, n_items = dataset.interactions_shape()

In [26]:
item_ids = np.arange(n_items)
model.predict(0,item_ids)

array([-1.23646808, -1.70065856,  5.37920761, -0.06361315,  2.27066755,
       -1.33141685,  2.66529179,  0.00688331, -0.29736784, -1.89530444,
       -0.31261554, -2.90992355,  1.85000801, -1.23716521, -0.84412533,
        2.6273644 ,  3.01566887,  0.74071002, -0.48012   ,  1.3650552 ,
        4.0676527 ,  2.18624139,  2.50508738, -2.50076103])

In [27]:
dataset.mapping()[1][1170544]

235546

In [28]:
preds = []
item_ids = np.arange(n_items)
for user_id in users:
    emb_id = dataset.mapping()[1][user_id]
    scores = model.predict(emb_id,item_ids)
    preds.append(scores)
    scores = []

In [31]:
from sklearn.metrics import roc_auc_score


def precision_at_k(model, ground_truth, k):
    """
    Measure precision at k for model and ground truth.

    Arguments:
    - lightFM instance model
    - sparse matrix ground_truth (no_users, no_items)
    - int k

    Returns:
    - float precision@k
    """

    ground_truth = ground_truth.tocsr()

    no_users, no_items = ground_truth.shape

    pid_array = np.arange(no_items, dtype=np.int32)

    precisions = []

    for user_id, row in enumerate(ground_truth):
        uid_array = np.empty(no_items, dtype=np.int32)
        uid_array.fill(user_id)
        predictions = model.predict(uid_array, pid_array, num_threads=4)

        top_k = set(np.argsort(-predictions)[:k])
        true_pids = set(row.indices[row.data == 1])

        if true_pids:
            precisions.append(len(top_k & true_pids) / float(k))

    return sum(precisions) / len(precisions)


def full_auc(model, ground_truth):
    """
    Measure AUC for model and ground truth on all items.

    Arguments:
    - lightFM instance model
    - sparse matrix ground_truth (no_users, no_items)

    Returns:
    - float AUC
    """

    ground_truth = ground_truth.tocsr()

    no_users, no_items = ground_truth.shape

    pid_array = np.arange(no_items, dtype=np.int32)

    scores = []

    for user_id, row in enumerate(ground_truth):
        uid_array = np.empty(no_items, dtype=np.int32)
        uid_array.fill(user_id)
        predictions = model.predict(uid_array, pid_array, num_threads=4)

        true_pids = row.indices[row.data == 1]

        grnd = np.zeros(no_items, dtype=np.int32)
        grnd[true_pids] = 1

        if len(true_pids):
            scores.append(roc_auc_score(grnd, predictions))

    return sum(scores) / len(scores)

In [ ]:
train_precision = precision_at_k(model, interactions, 10)




TypeError: ignored

In [42]:
test_precision = precision_at_k(model,users,10)

AttributeError: ignored

In [ ]:
train_auc = full_auc(model, interactions)
test_auc = full_auc(model, test)
    
print('Precision: %s, %s' % (train_precision, test_precision))
print('AUC: %s, %s' % (train_auc, test_auc))

In [37]:
train_precision

0.17804071272473626

In [ ]:
preds =np.array([np.array(predi) for predi in preds])

In [ ]:

preds = np.argsort(preds, axis=1)
preds = np.fliplr(preds)

In [ ]:
preds[0]

array([22,  8,  2, 21, 20, 16,  7, 15, 18,  4, 19,  3,  6, 13,  0, 23, 17,
       12,  1, 14, 11, 10,  9,  5])

In [ ]:
m1 = []
m2 = []
m3 = []
m4 = []
m5 = []
m6 = []
m7 = []
m8 = []
m9 = []
m10 = []
m11 = []
m12 = []
m13 = []
m14 = []
m15 = []
m16 = []
m17 = []

In [ ]:
final_month

,ncodpers,product,owned
26,657788,ind_cco_fin_ult1,1.0
50,657795,ind_cco_fin_ult1,1.0
76,657790,ind_cno_fin_ult1,1.0
84,657790,ind_ecue_fin_ult1,1.0
87,657790,ind_nom_pens_ult1,1.0
...,...,...,...
22354778,1166764,ind_cco_fin_ult1,1.0
22354802,1166763,ind_cco_fin_ult1,1.0
22354826,1166789,ind_cco_fin_ult1,1.0
22354850,1550586,ind_cco_fin_ult1,1.0


In [ ]:
print("Step 1/11")
#Get the target columns
labels_final_month = final_month[product_cols]
print("Step 2/11")
#Add ncodpers to the dataframe
labels_final_month['ncodpers'] = final_month.ncodpers
print("Step 3/11")
labels_final_month = labels_final_month.set_index("ncodpers")
print("Step 4/11")
stacked_labels_final_month = labels_final_month.stack()
print("Step 5/11")
filtered_labels_final_month = stacked_labels_final_month.reset_index()
print("Step 6/11")
filtered_labels_final_month.columns = ["ncodpers", "product", "owned"]
print("Step 7/11")
#Only select the rows where there is a new product.
filtered_labels_final_month = filtered_labels_final_month[filtered_labels_final_month["owned"] == 1]
print("Step 8/11")
#Merge with the training features.
print("Step 9/11")
final_month = filtered_labels_final_month.drop_duplicates(keep='last')
print("Step 10/11")
labels_final_month_final = final_month['product']

In [ ]:
final_month['product_#'] = labels_final_month_final.astype('category').cat.codes

In [ ]:
final_month.head()

,ncodpers,product,owned,product_#
26,657788,ind_cco_fin_ult1,1.0,2
50,657795,ind_cco_fin_ult1,1.0,2
76,657790,ind_cno_fin_ult1,1.0,4
84,657790,ind_ecue_fin_ult1,1.0,12
87,657790,ind_nom_pens_ult1,1.0,15


In [ ]:
#create a dictionary to store each product a customer already has
used_recommendation_products = {}
target_cols_all = np.array(product_cols)
#iterate through used_products and add each one to used_recommendation_products
for idx,row_val in final_month.iterrows():
    used_recommendation_products.setdefault(row_val['ncodpers'],[]).append(target_cols_all[row_val['product_#']])
    if len(used_recommendation_products) % 100000 == 0:
        print(len(used_recommendation_products))

100000
200000
300000
300000
400000
500000
600000
600000
600000
600000
600000
600000


In [ ]:
#users are the customer codes for the testing data.

target_cols_all = np.array(product_cols)
final_preds = []
#iterate through our model's predictions (pred) and add the 7 most recommended products that the customer does not have.
for idx,predicted in enumerate(preds):
    ids = users[idx]
    top_product = target_cols_all[predicted]
    used_products = used_recommendation_products.get(ids,[])
    new_top_product = []
    for product in top_product:
        if product not in used_products:
            new_top_product.append(product)
        if len(new_top_product) == 7:
            break
    final_preds.append(' '.join(new_top_product))
    if len(final_preds) % 100000 == 0:
        print(len(final_preds))

In [ ]:

submission = pd.DataFrame({'ncodpers':users,'added_products':final_preds})
#submission.to_csv('submission.csv',index=False)

In [ ]:
submission.head(10)

,ncodpers,added_products
0,15889,ind_recibo_ult1 ind_nomina_ult1 ind_ctop_fin_u...
1,1170544,ind_recibo_ult1 ind_ctop_fin_ult1 ind_tjcr_fin...
2,1170545,ind_recibo_ult1 ind_ctma_fin_ult1 ind_cno_fin_...
3,1170547,ind_recibo_ult1 ind_ecue_fin_ult1 ind_dela_fin...
4,1170548,ind_recibo_ult1 ind_nom_pens_ult1 ind_cno_fin_...
5,1170550,ind_recibo_ult1 ind_ctma_fin_ult1 ind_nom_pens...
6,1170552,ind_nom_pens_ult1 ind_nomina_ult1 ind_cno_fin_...
7,1170553,ind_recibo_ult1 ind_cno_fin_ult1 ind_nomina_ul...
8,1170555,ind_recibo_ult1 ind_nom_pens_ult1 ind_nomina_u...
9,1170557,ind_nom_pens_ult1 ind_recibo_ult1 ind_cno_fin_...


**Surprise SVD implementation**

In [8]:
product_cols = ['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1',
                'ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1','ind_ctop_fin_ult1',
                'ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_dela_fin_ult1','ind_deme_fin_ult1',
                'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_nom_pens_ult1',
                'ind_nomina_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1',
                'ind_recibo_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1']

In [12]:
train = pd.read_csv(data_path + "train_ver2.0.csv")
train = train.loc[train['fecha_dato']=='2016-05-28']

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,8,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
lastmonthlabels = train[product_cols]
lastmonthlabels['ncodpers'] = train.ncodpers #check if another last month
lastmonthlabels = lastmonthlabels.set_index('ncodpers')
stacked_lastmonthlabels = lastmonthlabels.stack()
filtered_lastmonthlabels = stacked_lastmonthlabels.reset_index()
filtered_lastmonthlabels.columns = ["ncodpers", "product", "is_owned"]
lastmonthlabels=[]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
stacked_lastmonthlabels

ncodpers                   
657640    ind_ahor_fin_ult1    0.0
          ind_aval_fin_ult1    0.0
          ind_cco_fin_ult1     0.0
          ind_cder_fin_ult1    0.0
          ind_cno_fin_ult1     0.0
                              ... 
1550586   ind_reca_fin_ult1    0.0
          ind_recibo_ult1      0.0
          ind_tjcr_fin_ult1    0.0
          ind_valo_fin_ult1    0.0
          ind_viv_fin_ult1     0.0
Length: 22354872, dtype: float64

In [7]:
pip install surprise

     |████████████████████████████████| 6.5MB 2.8MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1675758 sha256=2404eddb177e926e4c9508cbdd7e957901e589dfb3b4aff030a9b1cdee5b508e
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [5]:
from surprise import SVDpp
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import accuracy
from surprise.model_selection import train_test_split



In [14]:
reader = Reader(line_format='user item rating', rating_scale=(0,1))
reader

In [15]:
data = Dataset.load_from_df(filtered_lastmonthlabels, reader=reader)
data

In [16]:
trainset = data.build_full_trainset()
trainset

In [17]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.
        
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

In [18]:
from surprise.model_selection import KFold
from collections import defaultdict

In [1]:
kf = KFold(n_splits=5)
algo = SVDpp(verbose=True)

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))

NameError: ignored

In [11]:
test = pd.read_csv(data_path + "test_ver2.0.csv",nrows=40000)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
users = test.ncodpers

NameError: ignored

In [13]:
uid = users[0]
iid = product_cols[2]

In [1]:
pred =algo.predict(uid,iid)

NameError: ignored

In [ ]:
accuracy.rmse(pred, verbose=True)

In [15]:
from sklearn.metrics import confusion_matrix 

from sklearn.metrics import accuracy_score 

In [16]:
pred

Prediction(uid=15889, iid='ind_cco_fin_ult1', r_ui=None, est=0.7358079163624184, details={'was_impossible': False})

In [ ]:
#confusion_matrix(pred, dt_predicts) 
#accuracy_score(pred, dt_predicts) 

In [ ]:
preds =[]
user_preds = []
for user in users:
    for product in product_cols:
        user_preds.append(algo.predict(user,product)[3])
    preds.append(user_preds)
    user_preds = []

In [ ]:
preds =np.array([np.array(predi) for predi in preds])

In [ ]:
preds = np.argsort(preds, axis=1)
preds = np.fliplr(preds)

In [ ]:

final_month_training_cols = train
final_month_training_cols = final_month_training_cols.drop(product_cols, axis=1)

In [ ]:
#Get the target columns
labels_final_month = train[product_cols]

#Add ncodpers to the dataframe
labels_final_month['ncodpers'] = train.ncodpers

labels_final_month = labels_final_month.set_index("ncodpers")

stacked_labels_final_month = labels_final_month.stack()

filtered_labels_final_month = stacked_labels_final_month.reset_index()

filtered_labels_final_month.columns = ["ncodpers", "product", "newly_added"]

#Only select the rows where there is a new product.
filtered_labels_final_month = filtered_labels_final_month[filtered_labels_final_month["newly_added"] == 1]

#Merge with the training features.
#multiclass_final_month = filtered_labels_final_month.merge(final_month_training_cols, on="ncodpers", how="left")

#final_month = multiclass_final_month.drop_duplicates(multiclass_final_month, keep='last')

#labels_final_month_final = final_month['product']

#final_month_ncodpers = final_month.ncodpers
#Remove the columns that are not needed to train the model.
#final_month = final_month.drop(['ncodpers','newly_added','product'], axis = 1)

In [ ]:
multiclass_final_month

,ncodpers,product,newly_added,fecha_dato,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
0,657788,ind_cco_fin_ult1,1.0,2016-05-28,N,ES,H,42,2006-11-18,0.0,114,1.0,NaN,1,A,S,N,NaN,KBF,N,1.0,28.0,MADRID,1.0,132559.35,02 - PARTICULARES
1,657795,ind_cco_fin_ult1,1.0,2016-05-28,N,ES,V,44,2006-11-18,0.0,114,1.0,NaN,1,A,S,N,NaN,KAP,N,1.0,26.0,"RIOJA, LA",1.0,81399.57,02 - PARTICULARES
2,657790,ind_cno_fin_ult1,1.0,2016-05-28,N,ES,V,42,2006-11-18,0.0,114,1.0,NaN,1,A,S,N,NaN,KBF,N,1.0,48.0,BIZKAIA,1.0,NaN,02 - PARTICULARES
3,657790,ind_ecue_fin_ult1,1.0,2016-05-28,N,ES,V,42,2006-11-18,0.0,114,1.0,NaN,1,A,S,N,NaN,KBF,N,1.0,48.0,BIZKAIA,1.0,NaN,02 - PARTICULARES
4,657790,ind_nom_pens_ult1,1.0,2016-05-28,N,ES,V,42,2006-11-18,0.0,114,1.0,NaN,1,A,S,N,NaN,KBF,N,1.0,48.0,BIZKAIA,1.0,NaN,02 - PARTICULARES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240533,1166764,ind_cco_fin_ult1,1.0,2016-05-28,N,ES,V,23,2013-08-14,0.0,33,1.0,NaN,1,I,S,N,NaN,KHE,N,1.0,26.0,"RIOJA, LA",0.0,23334.99,03 - UNIVERSITARIO
1240534,1166763,ind_cco_fin_ult1,1.0,2016-05-28,N,ES,H,47,2013-08-14,0.0,33,1.0,NaN,1,A,S,N,NaN,KHE,N,1.0,50.0,ZARAGOZA,1.0,NaN,02 - PARTICULARES
1240535,1166789,ind_cco_fin_ult1,1.0,2016-05-28,N,ES,H,22,2013-08-14,0.0,33,1.0,NaN,1.0,I,S,N,NaN,KHE,N,1.0,50.0,ZARAGOZA,0.0,199592.82,03 - UNIVERSITARIO
1240536,1550586,ind_cco_fin_ult1,1.0,2016-05-28,N,ES,H,37,2016-05-13,1.0,0,1.0,NaN,NaN,NaN,S,N,NaN,NaN,N,1.0,28.0,MADRID,1.0,NaN,NaN


In [ ]:
labels_final_month_final = multiclass_final_month['product']


In [ ]:
final_month_ncodpers = multiclass_final_month.ncodpers
#Remove the columns that are not needed to train the model.
final_month = multiclass_final_month.drop(['ncodpers','newly_added','product'], axis = 1)

In [ ]:
labels_final_month_final_cat = labels_final_month_final.astype('category').cat.codes

In [ ]:
used_products = pd.DataFrame()
used_products['product'] = labels_final_month_final_cat
used_products['ncodpers'] = final_month_ncodpers
used_products = used_products.drop_duplicates(keep = 'last')

In [ ]:
#create a dictionary to store each product a customer already has
used_recommendation_products = {}
product_cols = np.array(product_cols)
#iterate through used_products and add each one to used_recommendation_products
for idx,row_val in used_products.iterrows():
    used_recommendation_products.setdefault(row_val['ncodpers'],[]).append(product_cols[row_val['product']])
    if len(used_recommendation_products) % 100000 == 0:
        print(len(used_recommendation_products))

100000
200000
300000
300000
400000
500000
600000
600000
600000
600000
600000
600000


In [ ]:
#test_ids are the customer codes for the testing data.
test_ids = np.array(pd.read_csv(data_path + "test_ver2.0.csv",nrows=40000,usecols=['ncodpers'])['ncodpers'])
target_cols_all = np.array(product_cols)
final_preds = []
#iterate through our model's predictions (pred) and add the 7 most recommended products that the customer does not have.
for idx,predicted in enumerate(preds):
    ids = test_ids[idx]
    top_product = target_cols_all[predicted]
    used_products = used_recommendation_products.get(ids,[])
    new_top_product = []
    for product in top_product:
        if product not in used_products:
            new_top_product.append(product)
        if len(new_top_product) == 7:
            break
    final_preds.append(' '.join(new_top_product))
    if len(final_preds) % 100000 == 0:
        print(len(final_preds))

In [ ]:
submission = pd.DataFrame({'ncodpers':test_ids,'added_products':final_preds})
#submission.to_csv('submission.csv',index=False)

In [ ]:
submission.head()

NameError: ignored

In [ ]:
def get_MAP(y_true, y_scores, k=7):
    print("Getting MAP scores.")
    # generating a numpy array product recommendations
    recs = (-y_scores).argsort()[:, :7]
    # initalising scores
    score = 0
    count = 0

    # calculate mean average precision for each user
    for i in range(y_true.shape[0]):
        count += 1
        # find column indexes for any products actually bought
        if np.argwhere(y_true[i] == True).shape[0] > 0:
            choices = np.argwhere(y_true[i] == True)[0]
        else:
            continue

        # selecting users recommendations
        top_k = recs[i]
        if choices.shape[0] == 0 or top_k.shape[0] == 0:
            continue

        else:
            # calculate metric to compare the actual choices with recommendations
            l_top_items = top_k.tolist()
            l_choices = choices.tolist()
            current = 0
            num_items_found = 0
            numerator = 0
            denominator = 0

            for guess in l_top_items:
                denominator += 1
                if guess in l_choices:
                    num_items_found += 1
                    numdef get_MAP(y_true, y_scores, k=7):

    print("Getting MAP scores.")

    # generating a numpy array product recommendations
    recs = (-y_scores).argsort()[:, :7]

    # initalising score
    score = 0
    count = 0

    # calculate mean average precision for each user
    for i in range(y_true.shape[0]):
        count += 1

        # find column indexes for any products actually bought
        if np.argwhere(y_true[i] == True).shape[0] > 0:
            choices = np.argwhere(y_true[i] == True)[0]

        else:
            continue

        # selecting users recommendations
        top_k = recs[i]
        if choices.shape[0] == 0 or top_k.shape[0] == 0:
            continue

        else:

            # calculate metric to compare the actual choices with recommendations
            l_top_items = top_k.tolist()
            l_choices = choices.tolist()
            current = 0
            num_items_found = 0
            numerator = 0
            denominator = 0

            for guess in l_top_items:
                denominator += 1

                if guess in l_choices:
                    num_items_found += 1
                    numerator += 1
                    current += numerator/denominator

            # increase score if at least 1 of the recommendations matched the actual choice
            if num_items_found > 0:
                score += current / num_items_found

    # return map score
    return score / counterator += 1
                    current += numerator/denominator
            # increase score if at least 1 of the recommendations matched the actual choice
            if num_items_found > 0:
                score += current / num_items_found
    # return map score
    return score / count

In [ ]:
map_score = get_MAP(np.array(Y_test), model.predict_proba(X_test))

print(map_score)

In [ ]:
from sklearn.metrics import confusion_matrix 

from sklearn.metrics import accuracy_score 



confusion_matrix(y_test, dt_predicts) 

accuracy_score(y_test, dt_predicts) 